# 2D / 3D Mapping

 ## Boreholes 2D mapping from files

### imports

In [1]:
import matplotlib.pyplot as plt
#from striplog import Component, Decor, Legend
#from core.omf import Borehole3D, striplog_legend_to_omf_legend 

import geopandas as gpd, pandas as pd
from shapely import wkt
import folium as fm
from folium import plugins
from utils.io import boreholes_from_files, striplog_from_text, export_gdf, read_gdf_file
from os import remove
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

### Data from a shapefile

In [2]:
shapef_dir = ROOT_DIR+"/CF_data/Data_UMONS/SIG/"
shp = gpd.read_file(shapef_dir+"Pilote_v8.0.shp")

**Rmq** : ici à quoi correspondent :

    - ID (tout est à 0!), 
    - Ref
    - Type

In [3]:
if 'Id' in shp.columns:
    shp=shp.drop(['Id'], axis=1)
    #shp.index.name='Id'
shp.head(2)

,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.891 122592.127)
1,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.447 122588.102)


In [4]:
pwd

'/home/yanathan/Projects/GSDMA/playground/TFE_test'

### Save location data in a Geopackage / GeoJson / csv

In [5]:
export_gdf(shp, 31370, 'tmp_files/test_loc.csv')

tmp_files/test_loc.csv has been saved !


In [6]:
cat 'tmp_files/test_loc.csv'

X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
152894.89100002544,122592.12713904586,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.8910000254 122592.1271390459)
152881.4470000254,122588.10213904455,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.4470000254 122588.1021390446)
152885.99500002537,122587.12713904679,2,1,217,3,1,1,1,1,0,OUI,POINT (152885.9950000254 122587.1271390468)
152883.7680000254,122589.75413904712,1,1,116,2,1,1,0,1,0,OUI,POINT (152883.7680000254 122589.7541390471)
152888.1280000254,122588.43613904715,2,1,214,2,1,1,1,1,0,OUI,POINT (152888.1280000254 122588.4361390471)
152886.2260000254,122591.65513904672,1,1,113,2,1,1,0,1,0,OUI,POINT (152886.2260000254 122591.6551390467)
152880.9800000254,122583.69313904643,2,1,223,3,1,1,1,1,1,OUI,POINT (152880.9800000254 122583.6931390464)
152883.65800002537,122585.62313904893,2,1,221,2,1,1,1,1,1,OUI,POINT (152883.6580000254 122585.6231390489)
152889.1520000254,122587.1761390483,1,1,118,2,1,1,0,1,0,OUI,POINT (152889.15200002

### 2D Map

-idée à explorer : fonction pour générer une carte des forages présents dans un projet

In [7]:
#Change CRS EPSG 31370 (Lambert 72) into EPSG 4326 (WGS 84)
shp = shp.to_crs(epsg=4326)
center=[shp.geometry.y.mean(), shp.geometry.x.mean()]

# Use a satellite map
tile_name = ["Satellite"]
tile_attr = ["Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"]
tile_url = ["https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"]

BH_map=fm.Map(location=center, tiles='OpenStreetMap', zoom_start=19, max_zoom=25, control_scale=True)

ch1 = fm.FeatureGroup(name='Boreholes')

for idx, row in shp.iterrows():
    fm.CircleMarker([row.geometry.y, row.geometry.x], popup=str(idx) , radius=0.1,
                     color='red', fill_color='red',
                    opacity=0.6
                    ).add_to(ch1)
    
#fm.GeoJson('BH_loc.json').add_to(fm_map)

mini_map = plugins.MiniMap(toggle_display=True, zoom_level_offset=-6)

#adding features to the base_map
fm.TileLayer(name=tile_name[0], tiles=tile_url[0], max_zoom=25, attr=tile_attr[0], control=True).add_to(BH_map)
ch1.add_to(BH_map)
fm.LayerControl().add_to(BH_map)
BH_map.add_child(mini_map)

#save in a file
BH_map.save('tmp_files/BH_location.html')
BH_map

### Reading location data from a csv, Geo(package/Json) file

In [8]:
file_dir=ROOT_DIR+'/playground/TFE_test/tmp_files/'

In [9]:
bh_loc=read_gdf_file(file_dir+'test_loc.csv')
bh_loc.head(2)

data EPSG (a number) ? : 31370


,X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,152894.891,122592.127139,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.891 122592.127)
1,152881.447,122588.102139,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.447 122588.102)


**Rmq** : la coordonnée Z est manquante dans les données de base !

### Create a boreholes positions list

In [10]:
bh_loc.loc[:2, ['X','Y','Ref', ]]

,X,Y,Ref
0,152894.891,122592.127139,111
1,152881.447,122588.102139,120
2,152885.995,122587.127139,217


In [11]:
x_list=bh_loc['X'].to_list()
y_list=bh_loc['Y'].to_list()
'Total: ', len(x_list), x_list

('Total: ',
 88,
 [152894.89100002544,
  152881.4470000254,
  152885.99500002537,
  152883.7680000254,
  152888.1280000254,
  152886.2260000254,
  152880.9800000254,
  152883.65800002537,
  152889.1520000254,
  152883.43300002534,
  152883.43300002534,
  152883.57100002535,
  152891.94900002552,
  152884.22600002537,
  152888.80200002543,
  152886.53400002536,
  152885.4830000254,
  152888.76800002545,
  152885.03158300993,
  152890.6620000255,
  152881.17100002535,
  152883.5060000254,
  152879.12900002534,
  152885.8040000254,
  152884.55900002542,
  152886.79600002544,
  152881.57883449996,
  152886.77900002545,
  152881.7110000254,
  152893.79900002544,
  152886.1190000254,
  152891.25200002544,
  152890.82431275706,
  152892.8790000254,
  152885.19800002538,
  152888.16400002537,
  152890.42400002546,
  152888.12300002543,
  152891.16600002543,
  152893.33600002548,
  152895.43100002548,
  152888.20900002544,
  152891.32900002546,
  152889.47000002547,
  152887.7500000254,
  15288

## Borehole 3D mapping

### Build Borehole3D objects

Puisqu'on ne dispose que de la localisation des forages (pas d'infos sur la géométrie et le "contenu" des forages), l'idée est de créer **une géométrie et un contenu par défaut** pour chaque forage, **tout en respectant les positions**, puis les enregistrer dans la BD afin de pouvoir les afficher tous dans l'environnement 3D. La mise à jour des données manquantes se fera plus tard.

In [12]:
remove('tmp_files/tfe_orm_db.db')

Create the db engine and the database structure

In [13]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)
Base.metadata.create_all(engine)

2021-02-20 14:30:52,733 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-20 14:30:52,734 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:52,735 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-20 14:30:52,736 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:52,737 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-02-20 14:30:52,737 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:52,738 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-02-20 14:30:52,738 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:52,739 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-02-20 14:30:52,739 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:52,740 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-02-20 14:30:52,740 INFO sqlalchemy.engine.base.Engine ()
2021-02-20

creation of the Boreholes from las, csv files

In [14]:
lexicon = Lexicon.default() #to do : create our own lexicon

In [15]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las'}#, 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

NB: **Problème ici, car on a attribué 'F1' comme nom du forage, donc écrasement dans la BD lors de l'ajout des autres BH**

In [16]:
boreholes, components= boreholes_from_files(borehole_dict, x=x_list, y=y_list, verbose=0)

In [17]:
len(boreholes), boreholes[0]

(88, <core.orm.BoreholeOrm at 0x7fd68769aee0>)

Create the session and the project

In [18]:
Session = sessionmaker(bind=engine)
session = Session()

p = Project(session)
p.add_components(components)

2021-02-20 14:30:53,262 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:30:53,263 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-20 14:30:53,263 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:53,266 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-02-20 14:30:53,267 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-02-20 14:30:53,268 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-20 14:30:53,273 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:30:53,274 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-20 14:30:53,274 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [19]:
for bh in boreholes:
    p.add_borehole(bh)

2021-02-20 14:30:53,286 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-20 14:30:53,287 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 152894.89100002544, 122592.12713904586), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 152894.89100002544, 122592.12713904586), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 152894.89100002544, 122592.12713904586), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 152894.89100002544, 122592.12713904586), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 152894.89100002544, 122592.12713904586), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 152894.89100002544, 122592.12713904586), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 152894.89100002544, 122592.12713904586), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 152894.89100002544, 122592.12713904586)  ... displaying 10 of 

2021-02-20 14:30:53,337 INFO sqlalchemy.engine.base.Engine (13,)
2021-02-20 14:30:53,339 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,339 INFO sqlalchemy.engine.base.Engine (14,)
2021-02-20 14:30:53,340 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,341 INFO sqlalchemy.engine.base.Engine (15,)
2021-02-20 14:30:53,343 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:30:53,386 INFO sqlalchemy.engine.base.Engine (34,)
2021-02-20 14:30:53,387 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,387 INFO sqlalchemy.engine.base.Engine (35,)
2021-02-20 14:30:53,388 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,389 INFO sqlalchemy.engine.base.Engine (36,)
2021-02-20 14:30:53,390 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:30:53,470 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:30:53,470 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-20 14:30:53,471 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:30:53,471 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-20 14:30:53,472 INFO sqlalchemy.engine.base.Engine ('F2',)
2021-02-20 14:30:53,473 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 

2021-02-20 14:30:53,510 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,511 INFO sqlalchemy.engine.base.Engine (69,)
2021-02-20 14:30:53,514 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:30:53,514 INFO sqlalchemy.engine.base.Engine (70,)
2021-02-20 14:30:53,515 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Position

...

2021-02-20 14:31:10,508 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,509 INFO sqlalchemy.engine.base.Engine (4308,)
2021-02-20 14:31:10,510 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,510 INFO sqlalchemy.engine.base.Engine (4309,)
2021-02-20 14:31:10,513 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Posi

2021-02-20 14:31:10,563 INFO sqlalchemy.engine.base.Engine (4328,)
2021-02-20 14:31:10,565 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,565 INFO sqlalchemy.engine.base.Engine (4329,)
2021-02-20 14:31:10,569 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,569 INFO sqlalchemy.engine.base.Engine (4330,)
2021-02-20 14:31:10,571 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "

2021-02-20 14:31:10,616 INFO sqlalchemy.engine.base.Engine (4349,)
No legend given or incorrect format ! A default legend will be used
Borehole geometry created successfully !
2021-02-20 14:31:10,662 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-02-20 14:31:10,663 INFO sqlalchemy.engine.base.Engine ('F88',)
2021-02-20 14:31:10,665 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-20 14:31:10,665 INFO sqlalchemy.engine.base.Engine ((4350, 14.3, 14.3, 14.3, 152884.7420000254, 122588.50313904788), (4351, 51.24117647058824, 51.24117647058824, 51.24117647058824, 152884.7420000254, 122588.50313904788), (4352, 51.24117647058824, 51.24117647058824, 51.24117647058824, 152884.7420000254, 122588.50313904788), (4353, 54.81764705882354, 54.81764705882354, 54.81764705882354, 152884.7420000254, 122588.50313904788), (4354, 54.81764705882354, 54.81764705882354, 54.81764705882354, 152884.7420000

2021-02-20 14:31:10,717 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,717 INFO sqlalchemy.engine.base.Engine (4363,)
2021-02-20 14:31:10,720 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,720 INFO sqlalchemy.engine.base.Engine (4364,)
2021-02-20 14:31:10,721 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Posi

2021-02-20 14:31:10,770 INFO sqlalchemy.engine.base.Engine (4383,)
2021-02-20 14:31:10,772 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,773 INFO sqlalchemy.engine.base.Engine (4384,)
2021-02-20 14:31:10,774 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:31:10,775 INFO sqlalchemy.engine.base.Engine (4385,)
2021-02-20 14:31:10,778 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "

In [20]:
p.commit()

2021-02-20 14:31:10,859 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
p.boreholes[0].intervals[0].description

2021-02-20 14:31:10,864 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:31:10,865 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-02-20 14:31:10,866 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-20 14:31:10,867 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-20 14:31:10,868 INFO sqlalchemy.engine.base.Engine ('F1',)


'Anhydrite'

In [22]:
session.close()

2021-02-20 14:31:10,874 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [23]:
p.plot3d(x3d=True, radius=0.5)